In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_data_path = '/content/drive/MyDrive/train_total/train_total_02'

train_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',subset='training',shuffle=True)




Found 17150 images belonging to 49 classes.


In [4]:
validate_data_path = '/content/drive/MyDrive/validation_total_2'
validate_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validate_datagen.flow_from_directory(
    validate_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',shuffle=True) # Use 'validation' subset for validation data

Found 4900 images belonging to 49 classes.


In [5]:
test_data_path = '/content/drive/MyDrive/test_total'
#validate_data_path = '/content/drive/MyDrive/Groundnut_Leaf_dataset/test'
test_datagen = ImageDataGenerator(rescale=1./255)
#validate_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',shuffle=True)

Found 4910 images belonging to 49 classes.


In [6]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for classes present in the training and validation datasets

0 : 01_Aayushi_Gautam
1 : 02_Abishek_Banerjee
2 : 03_Kaushik_Agarwal
3 : 04_Niyaj_Attar
4 : 05_Avesh_Bhati
5 : 06_Ayan_Datta
6 : 07_Ayush_Mitra
7 : 08_Omkar_Bankar
8 : 09_Prajakta_Bhand
9 : 10_Sahil_Bhosale
10 : 11_Bhuvan_Mangat
11 : 12_tasha_cardozo
12 : 13_Chhavi_Baliyan
13 : 14_Piyush_Dambhare
14 : 15_Khyati_Desai
15 : 16_Priyal_Dighe
16 : 17_Dilpreet_Kaur
17 : 18_Divyanshu_Bisht
18 : 19_Ekta_Chaware
19 : 20_Ashwin_Gadge
20 : 21_Idhika_Vaidya
21 : 22_Aman_Jawalekar
22 : 23_Juned_Manyar
23 : 25_Prachi_Kamble
24 : 26_pranav_katariya
25 : 27_Sourav_Khot
26 : 28_Kinjal_Bandopadhyay
27 : 29_Mallika_Manu
28 : 30_nadar_janice
29 : 31_Santosh_Nagarale
30 : 33_Omkar_Pol
31 : 34_Dhruvi_Panchal
32 : 35_Raj_Aryan
33 : 37_Reetam_Ghosh
34 : 38_Ritwik_Dubey
35 : 39_Raghuram_Sabavath
36 : 40_Sahil_Raj
37 : 41_Sampurna_Mondal
38 : 43_Saranya_Roy
39 : 44_Gaurav_Sharma
40 : 45_Shubham_Shedekar
41 : 46_shivani_shukla
42 : 47_Sh

In [ ]:
inp=Input((224,224,3))

def inception_block(x,base_channels=16):
  conv_1=Conv2D(base_channels*2,1,1,activation='relu')(x)

  conv_2_a=Conv2D(base_channels*2,1,1,activation='relu')(x)
  conv_2_b=Conv2D(base_channels*2,3,1,padding='same',activation='relu')(conv_2_a)

  conv_3_a=Conv2D(base_channels,1,1,activation='relu')(x)
  conv_3_b=Conv2D(base_channels,5,1,padding='same',activation='relu')(conv_3_a)

  mpool_4_a=MaxPool2D(3,1,padding='same')(x)
  conv_4_b=Conv2D(base_channels,1,1,activation='relu')(mpool_4_a)

  return Concatenate(axis=-1)([conv_1,conv_2_b,conv_3_b,conv_4_b])

block_1=inception_block(inp)
block_2=inception_block(block_1,base_channels=8)
block_3=inception_block(block_2,base_channels=8)
#block_4=inception_block(block_3,base_channels=8)


gap=GlobalAveragePooling2D()(block_3)
flatn=Flatten()(gap)
output=Dense(49,activation="softmax")(flatn)

model=Model(inputs=inp,outputs=output)

model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 224, 224, 32)         128       ['input_1[0][0]']             
                                                                                                  
 conv2d_3 (Conv2D)           (None, 224, 224, 16)         64        ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 224, 224, 3)          0         ['input_1[0][0]']             
 D)                                                                                           

In [ ]:
es_callback = EarlyStopping(monitor='val_loss', mode='auto',patience=2)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.01),loss=keras.losses.categorical_crossentropy,metrics=["accuracy"])

In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=validation_generator,
                       callbacks=[es_callback])

Epoch 1/10
269/269 [==============================] - 8633s 32s/step - loss: 3.7021 - accuracy: 0.0446 - val_loss: 3.0011 - val_accuracy: 0.1878
Epoch 2/10
269/269 [==============================] - 265s 984ms/step - loss: 2.2279 - accuracy: 0.3593 - val_loss: 1.6692 - val_accuracy: 0.4978
Epoch 3/10
269/269 [==============================] - 261s 971ms/step - loss: 1.3178 - accuracy: 0.6161 - val_loss: 1.1914 - val_accuracy: 0.6862
Epoch 4/10
269/269 [==============================] - 262s 974ms/step - loss: 0.9925 - accuracy: 0.7089 - val_loss: 0.9554 - val_accuracy: 0.7346
Epoch 5/10
269/269 [==============================] - 264s 979ms/step - loss: 0.7830 - accuracy: 0.7716 - val_loss: 0.8514 - val_accuracy: 0.7337
Epoch 6/10
269/269 [==============================] - 264s 982ms/step - loss: 0.6678 - accuracy: 0.8038 - val_loss: 0.8101 - val_accuracy: 0.7935
Epoch 7/10
269/269 [==============================] - 298s 1s/step - loss: 0.5669 - accuracy: 0.8332 - val_loss: 0.5974 - val

In [ ]:
model.save('/content/drive/MyDrive/attendnce_final_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.evaluate(test_generator, batch_size=512)

77/77 [==============================] - 1505s 20s/step - loss: 0.4756 - accuracy: 0.8436


[0.4755784273147583, 0.8435845375061035]

In [ ]:
loaded_model = load_model('/content/drive/MyDrive/attendnce_final_model.h5')

In [ ]:
es_callback = EarlyStopping(monitor='val_loss', mode='auto',patience=2)

In [ ]:
loaded_model.compile(optimizer=keras.optimizers.Adam(lr=0.01),loss=keras.losses.categorical_crossentropy,metrics=["accuracy"])

In [ ]:
history = loaded_model.fit(train_generator, epochs=10, validation_data=validation_generator,
                       callbacks=[es_callback])

KeyboardInterrupt: ignored

In [ ]:
loaded_model.save('/content/drive/MyDrive/attendnce_final_model_01.h5')

In [7]:
loaded_model = load_model('/content/drive/MyDrive/attendnce_final_model_01.h5')

In [8]:
es_callback = EarlyStopping(monitor='val_loss', mode='auto',patience=2)

In [9]:
loaded_model.compile(optimizer=keras.optimizers.Adam(lr=0.1),loss=keras.losses.categorical_crossentropy,metrics=["accuracy"])

In [ ]:
history = loaded_model.fit(train_generator, epochs=4, validation_data=validation_generator,
                       callbacks=[es_callback])

Epoch 1/4
268/268 [==============================] - 7715s 29s/step - loss: 0.2905 - accuracy: 0.9180 - val_loss: 0.3288 - val_accuracy: 0.9149
Epoch 2/4
268/268 [==============================] - 226s 842ms/step - loss: 0.2867 - accuracy: 0.9168 - val_loss: 0.2838 - val_accuracy: 0.9194
Epoch 3/4
268/268 [==============================] - ETA: 0s - loss: 0.2592 - accuracy: 0.9251

In [ ]:
loaded_model.save('/content/drive/MyDrive/attendnce_final_model_02.h5')

In [ ]:
plt.plot(history.history['loss'],label="Training Loss")
plt.plot(history.history['val_loss'],label="Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Traing & Validation loss curve')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],label="Training accuracy")
plt.plot(history.history['val_accuracy'],label="Validation accuracy")
plt.xlabel('Epochs')
plt.ylabel('Traing & Validation accuracy curve')
plt.legend()
plt.show()